In [1063]:
import spacy
from spacy import displacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

STOP_WORDS.add("able")
if 'make' in STOP_WORDS:
    STOP_WORDS.remove("make")
    

doc = nlp("nurse is able to check patient info on the system")
# doc = nlp("Users should be able to update credit card info on payment screen, add items to carts, and pay for items.")

displacy.render(doc, style="dep", jupyter=True)
for tok in doc:
  print(tok.text, "...", tok.pos_, "...", tok.dep_)

new_doc = []
for tok in doc:
    new_doc.append(tok.lemma_)
new_doc = ' '.join(new_doc)
new_doc = nlp(new_doc)
print(new_doc)


c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


nurse ... NOUN ... nsubj
is ... AUX ... ROOT
able ... ADJ ... acomp
to ... PART ... aux
check ... VERB ... xcomp
patient ... ADJ ... amod
info ... NOUN ... dobj
on ... ADP ... prep
the ... DET ... det
system ... NOUN ... pobj
nurse be able to check patient info on the system


In [1064]:

df = pd.DataFrame(columns=['text', 'pos', 'tag', 'dep'])
for token in new_doc:
    df = df.append({'text': token.text, 'pos': token.pos_, 'tag': token.tag_, 'dep': token.dep_}, ignore_index=True)
print(df)

      text   pos tag    dep
0    nurse  NOUN  NN  nsubj
1       be   AUX  VB   ROOT
2     able   ADJ  JJ  acomp
3       to  PART  TO    aux
4    check  VERB  VB  xcomp
5  patient   ADJ  JJ   amod
6     info  NOUN  NN   dobj
7       on   ADP  IN   prep
8      the   DET  DT    det
9   system  NOUN  NN   pobj


C:\Users\Administrator\AppData\Local\Temp\ipykernel_14816\952258587.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'text': token.text, 'pos': token.pos_, 'tag': token.tag_, 'dep': token.dep_}, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14816\952258587.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'text': token.text, 'pos': token.pos_, 'tag': token.tag_, 'dep': token.dep_}, ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_14816\952258587.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'text': token.text, 'pos': token.pos_, 'tag': token.tag_, 'dep': token.dep_}, ignore_index=True)
C:\Users\Administrator\AppData\Loc

In [1065]:
for token in doc:
    ancestors = [t.text for t in token.ancestors]
    children = [t.text for t in token.children]
    print(token.text, "\t", 
          token.pos_, "\t", token.dep_, "\t", 
          ancestors, "\t", children)

nurse 	 NOUN 	 nsubj 	 ['is'] 	 []
is 	 AUX 	 ROOT 	 [] 	 ['nurse', 'able']
able 	 ADJ 	 acomp 	 ['is'] 	 ['check']
to 	 PART 	 aux 	 ['check', 'able', 'is'] 	 []
check 	 VERB 	 xcomp 	 ['able', 'is'] 	 ['to', 'info']
patient 	 ADJ 	 amod 	 ['info', 'check', 'able', 'is'] 	 []
info 	 NOUN 	 dobj 	 ['check', 'able', 'is'] 	 ['patient', 'on']
on 	 ADP 	 prep 	 ['info', 'check', 'able', 'is'] 	 ['system']
the 	 DET 	 det 	 ['system', 'on', 'info', 'check', 'able', 'is'] 	 []
system 	 NOUN 	 pobj 	 ['on', 'info', 'check', 'able', 'is'] 	 ['the']


In [1066]:
actor = []
isFirst = True

for tok in nlp(new_doc):
    if (tok.tag_ == "NN" or tok.tag_ == "NNS" or tok.tag_ == "NNP") and isFirst == True:
        isFirst = False
        actor.append(tok.text)

print(actor)

['nurse']


In [1067]:
def get_subject_phrase(doc):
    for token in doc:
        if ("subj" in token.dep_):
            subtree = list(token.subtree)
            start = subtree[0].i
            end = subtree[-1].i + 1
            return doc[start:end]

In [1068]:
def get_object_phrase(doc):
    for token in doc:
        if ("dobj" in token.dep_):
            subtree = list(token.subtree)
            for i in range(len(subtree)):
                if subtree[i].pos_ == 'ADP':
                    middle = subtree[i].i
            start = subtree[0].i
            end = subtree[-1].i + 1
            
            for tok in token.ancestors:
                if tok.pos_ == "VERB":
                    method = tok
                break;
            
            method = str(method) + " " + str(doc[start:middle])
            method = nlp(method)

            return [method, doc[(middle + 1):end]]

In [1069]:
def get_class_after_preposition(doc):
    terms = []
    for i in range(len(doc)):
        if doc[i].text == 'of' or doc[i].text == 'for' or doc[i].text == 'to':
            if doc[i+1].pos_ == 'NOUN':
                for tok in doc[i].ancestors:
                    if tok.pos_ == "VERB":
                        terms.append([tok])
                    break;
                terms[-1].append(doc[i+1])
                
    return terms

In [1070]:
subject= get_subject_phrase(new_doc)
obt = get_object_phrase(new_doc)
other_classes = get_class_after_preposition(new_doc)
print("Subject:", subject)
print("Direct object:", obt)
print("Other classes:", other_classes)

Subject: nurse
Direct object: [check patient info, the system]
Other classes: []


In [1071]:
classes = []
# if get_subject_phrase(new_doc).text not in actor:
#         classes.append(subject)
if obt != None:
        classes.append(obt)
for term in other_classes:
        classes.append(term)

print("Extracted classes are as follows.")
print(classes)

Extracted classes are as follows.
[[check patient info, the system]]


In [1072]:
tmp_part = classes
classes = []
for part in tmp_part:
    temp_part = []
    for term in part:
        temp_part.append(term.text.replace(' ', '_'))
    classes.append(temp_part)
print(classes)
            

[['check_patient_info', 'the_system']]


In [1073]:
f = open("classUML.puml", "w")
f.write("@startuml\n")
f.write("hide circle\n")
f.write("skinparam classFontStyle bold\n")

for i in range(len(classes)):
    f.write("class " + str(classes[i][1]) + "{\n")
    f.write(str(classes[i][0]) + "()" + "\n")
    f.write("}\n")

f.write("@enduml")

7